In [22]:
import numpy as np
from numpy.linalg import inv
import cv2
import glob
from sklearn.preprocessing import normalize
import time, pandas

In [23]:
def writePicture(filename, image, algorithm):
    filename = filename.replace("input", "result/" + algorithm)
    filename = filename.replace("in", "out")
    cv2.imwrite(filename, image);

In [24]:
# Basic Motion Detection

def d(frame, bs):
    return ((frame[:,:,0] - bs[:,:,0])**2 + (frame[:,:,1] - bs[:,:,1])**2 + (frame[:,:,2] - bs[:,:,2])**2)

def basicMotionDetetcion(alfa, t, frame, bs):
    b = (1-alfa)*bs + alfa*frame
    return b, (d(frame, bs) > t)*1*255

def bmd(images, filenames):

    alfa = 10**-3
    t = 5000
    bs = np.uint8(images[0])

    for i in range(1,len(images)):
        img = images[i]
        frame = np.uint8(img)
        bs, fgmask = basicMotionDetetcion(alfa, t, frame, bs)
        fgmask = np.uint8(fgmask)
        cv2.imshow('Basic Motion Detection', fgmask)
        writePicture(filenames[i], fgmask, "bmd")
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break

    cv2.destroyAllWindows()

In [25]:
# MOG2 -> Gaussian Mixture based foreground/background detecton algorithm 

def mog2(images, filenames):

    b0 = images[0]
    fgbg = cv2.createBackgroundSubtractorMOG2()

    for i in range(1,len(images)):
        frame = images[i]
        fgmask = fgbg.apply(frame)
        cv2.imshow('MOG2',fgmask)
        writePicture(filenames[i], fgmask, "mog2")
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break

    cv2.destroyAllWindows()

In [26]:
# KNN -> K Nearest Neighbours

def knn(images, filenames):

    b0 = images[0]
    fgbg = cv2.createBackgroundSubtractorKNN()

    for i in range(1,len(images)):
        frame = images[i]
        fgmask = fgbg.apply(frame)
        cv2.imshow('frame', fgmask)
        writePicture(filenames[i], fgmask, "knn")
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break

    cv2.destroyAllWindows()

In [27]:
testnames = ["dataset/test1/", "dataset/test2/", "dataset/test3/"]

for t in range(0, len(testnames)):
    # Ucitvanje slika za jedan test primer
    filenames = glob.glob(testnames[t] + "input/*.jpg")
    filenames.sort()
    
    images = [cv2.imread(img) for img in filenames]

    # Pozivanje algoritama za izdvajanje pozadine od pomerajucih objekata
    bmd(images, filenames)
    mog2(images, filenames)
    knn(images, filenames)

In [29]:
def eval(algoritam, filenames):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    
    for i in range(1, len(filenames)):
        resFilename = filenames[i].replace("input", "result/" + algoritam)
        resFilename = resFilename.replace("in", "out")
        gtFilename = filenames[i].replace("input", "ground_truth")
        gtFilename = gtFilename.replace("in", "gt")
        gtFilename = gtFilename.replace("jpg", "png")
       
        resImage = cv2.imread(resFilename)
        resImage = cv2.cvtColor(resImage,cv2.COLOR_BGR2GRAY)  

        gtImage = cv2.imread(gtFilename)
        gtImage = cv2.cvtColor(gtImage,cv2.COLOR_BGR2GRAY)  
        
        for j in range(0, resImage.shape[0]):
            for k in range(0, resImage.shape[1]):
                if resImage[j][k] != 0 and gtImage[j][k] != 0:
                    tp = tp + 1
                elif resImage[j][k] == 0 and gtImage[j][k] == 0:
                    tn = tn + 1
                elif resImage[j][k] != 0:
                    fp = fp + 1
                else:
                    fn = fn + 1
          
    prec = tp / (tp + fp)       
    recall = tp / (tp + fn)
    return prec, recall

In [30]:
testnames = ["dataset/test1/", "dataset/test2/", "dataset/test3/"]

bmdScore = []
mog2Score = []
knnScore = []

for t in range(0, len(testnames)):
    # Ucitvanje imena slika
    filenames = glob.glob(testnames[t] + "input/*.jpg")
    filenames.sort()
    

    # Evaluacija algoritama za izdvajanje pozadine od pomerajucih objekata
    prec, recall = eval("bmd", filenames)
    bmdScore.append((prec, recall))
    
    prec, recall = eval("mog2", filenames)
    mog2Score.append((prec, recall))
    
    prec, recall = eval("knn", filenames)
    knnScore.append((prec, recall))

print(bmdScore)
print(mog2Score)
print(knnScore)

[(0.9408605330803484, 0.4527993409838972), (0.47571929507812877, 0.7833279913785495), (0.9822120712674162, 0.9435565641183505)]
[(0.4236645083946855, 0.9413660169957543), (0.18202847590309232, 0.8926220636668106), (0.897246847987152, 0.7183750636783908)]
[(0.5218184050914061, 0.9156954515348574), (0.2739729504486401, 0.9024930670717262), (0.887228556007916, 0.906838854851408)]
